In [1]:
%load_ext rich
%load_ext ensurewd

In [2]:
from pathlib import Path

import pandas as pd
import polars as pl
from torchvision import models

from utils.config import get_conf
from utils.features_extract import (
    get_images_features,
    get_transformers,
)
from utils.func import get_path_list
from utils.models import VGG16Extractor

In [3]:
# path to dataset
TRAIN_PATH = Path('../data/raw/histopathologic-cancer-detection/train/')
TEST_PATH = Path('../data/raw/histopathologic-cancer-detection/test/')

FEATURES_MATRIX_CSV = Path(
    '../data/raw/histopathologic-cancer-detection/feature_matrix_train.csv',
)

TRAIN_LABELS_PATH_CSV = Path(
    '../data/raw/histopathologic-cancer-detection/train_labels.csv'
)

CONF_PATH = Path('../data/conf.toml')

dataset_conf = get_conf(conf_path=CONF_PATH)

histopathologic_cancer_conf = dataset_conf['histopatological_cancer_detection']
histopathologic_cancer_conf

{'features_matrix': ' '}

In [4]:
# train and test path list

train_images_path_list = get_path_list(TRAIN_PATH)
test_images_path_list = get_path_list(TEST_PATH)

### Using a custom model to extract images features

In [5]:
# getting model
vgg16 = models.vgg16(weights=True)

/home/smrenato/repo/eda_histopathologic_cancer_detect/.venv/lib64/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# building our custom model
device = 'cuda'  # cpu
features_extractor_model = VGG16Extractor(vgg16).to(device)

#### Building feature matrix

In [7]:
# transformer
transformeres = get_transformers()
transformeres


Compose(
    CenterCrop(size=(32, 32))
    Resize(size=448, interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
)

In [8]:
# get a list of feature from train dataset

img_features = get_images_features(
    features_extractor_model,
    train_images_path_list[:10],  # using just 10, because it'll take a while
    transformeres,
)
img_features[0], img_features[0].shape


(
    tensor([[ 0.2443, -0.2892, -2.0232,  ..., -1.1370,  0.0349, -1.2414]],
       device='cuda:0'),
    torch.Size([1, 4096])
)